# Ingestion
This notebook parses data from XXX (need detail here) using Google Vertex AI Generative AI.  It then uses Generative AI to create Neo4j Cypher queries which write the data to a Neo4j database.

## Setup
This notebook should be run within Vertex AI Workbench.  Be sure to select "single user" when starting a managed notebook to run this.  Otherwise the auth won't allow access to the preview.  

First we need to install the latest libraries for Generative AI.

In [ ]:
!pip install --user google-cloud-aiplatform --upgrade

You will need to restart the kernel after the pip install completes.

In [2]:
# Note, you will need to set your project_id
project_id = 'neo4jbusinessdev'
location = 'us-central1'

In [3]:
import vertexai
vertexai.init(project=project_id, location='us-central1')

## Data Cleansing

Now, let's define a function that can help clean the input data. The data refers to some figures like scanned images. We don't have them and so we will remove any such references.

In [10]:
import re

def clean_text(text):
  clean = "\n".join([row for row in text.split("\n")])
  clean = re.sub(r'\(fig[^)]*\)', '', clean, flags=re.IGNORECASE)
  return clean

## Prompt Definition

Let's take this case sheet and extract entities and relations using LLM

This is a helper function to talk to the LLM with our prompt and text input

In [110]:
def run_text_model(
    project_id: str,
    model_name: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    prompt: str,
    location: str = "us-central1",
    tuned_model_name: str = "",
    ) :
    """Text Completion Use a Large Language Model."""
    vertexai.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained(model_name)
    if tuned_model_name:
      model = model.get_tuned_model(tuned_model_name)
    response = model.predict(
        prompt,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,)
    return response.text

This is a simple prompt to start with. If the processing is very complex, you can also chain the prompts as and when required. I am going to use a single prompt here that will extract the text strictly as per the Entities and Relationships defined. This is a simplification. 
In the real scenario, especially with medical records, you have to leverage on Domain experts to define the Ontology systematically and capture the important information. You should also be mindful of following the relevant regulations around handling health records,

Instead of one single large model, you can also consider chaining a number of smaller ones as per your needs.

Let's run our completion task with our LLM

In [12]:
def extract_entities_relationships(prompt, tuned_model_name):
    try:
        res = run_text_model(project_id, "text-bison@001", 0, 1024, 0.8, 40, prompt, location, tuned_model_name)
        return res
    except Exception as e:
        print(e)
    

In [94]:
person_prompt_tpl="""From the Curriculum Vitae text for a job aspirant below, extract Entities strictly as instructed below
1. First, look for this Entity type in the text and generate as comma-separated format similar to entity type.
   `id` property of each entity must be alphanumeric and must be unique among the entities. You will be referring this property to define the relationship between entities. NEVER create new entity types that aren't mentioned below. Document must be summarized and stored inside Person entity under `description` property
    Entity Types:
    label:'Person',id:string,role:string,description:string //Person Node
2. Description property should be the text summary and not more than 100 characters
3. If you cannot find any information on the entities & relationships above, it is okay to return empty value. DO NOT create fictious data
4. Do NOT create duplicate entities
5. IMPORTANT: Restrict output within the token limit

Example Output Format:
{
    "entities": [{"label":"Person","id":"person1","role":"Prompt Developer","description":"Prompt Developer with more than 30 years of LLM experience"}]
}

Question: Now, extract the Person for the text below -
$ctext

Answer:
"""

In [95]:
postion_prompt_tpl="""From the Curriculum Vitae text for a job aspirant below, extract Entities & relationships strictly as instructed below
1. First, look for these Entity types in the text and generate as comma-separated format similar to entity type.
   `id` property of each entity must be alphanumeric and must be unique among the entities. You will be referring this property to define the relationship between entities. NEVER create new entity types that aren't mentioned below. You will have to generate as many entities as needed as per the types below:
    Entity Types:
    label:'Position',id:string,title:string,location:string,startDate:string,endDate:string,url:string //Position Node
    label:'Company',id:string,name:string //Company Node
2. Next generate each relationships as triples of head, relationship and tail. To refer the head and tail entity, use their respective `id` property. Relationship property should be mentioned within brackets as comma-separated. They should follow these relationship types below. You will have to generate as many relationships as needed as defined below:
    Relationship types:
    position|AT_COMPANY|company
3. If you cannot find any information on the entities & relationships above, it is okay to return empty value. DO NOT create fictious data
4. Do NOT create duplicate entities
5. IMPORTANT: Restrict output within the token limit
 
Example Output Format:
{
    "entities": [{"label":"Position","id":"position1","title":"Software Engineer","location":"Singapore",startDate:"2021-01-01",endDate:"present"},
    {"label":"Position","id":"position2","title":"Senior Software Engineer","location":"Mars",startDate:"2020-01-01",endDate:"2020-12-31"},
    {label:"Company",id:"company1",name:"Neo4j Singapore Pte Ltd"},
    {"label":"Company","id":"company2","name":"Neo4j Mars Inc"}
    ],
    "relationships": ["position1|AT_COMPANY|company1","position2|AT_COMPANY|company2"]
}

Question: Now, extract entities & relationships as mentioned above for the text below -
$ctext

Answer:
"""

In [96]:
skill_prompt_tpl="""From the Curriculum Vitae text below, extract Entities strictly as instructed below
1. First, look for Skill Entities in the text and generate as comma-separated format similar to entity type.
   `id` property of each entity must be alphanumeric and must be unique among the entities. You will be referring this property to define the relationship between entities. NEVER create new entity types that aren't mentioned below:
    Entity Types:
    label:'Skill',id:string,name:string,level:string //Skill Node
2. DO NOT create fictious data
3. Restrict to only 30 Skills //Figure out which skill is most used based on the experience and expertise

Example Output Format:
{
    "entities": [{"label":"Skill","id":"skill1","name":"Neo4j","level":"expert"},{"label":"Skill","id":"skill2","name":"Pytorch","level":"expert"}]
}

Question: Now, extract entities as mentioned above for the text below -
$ctext

Answer:
"""

In [97]:
edu_prompt_tpl="""From the Curriculum Vitae text for a job aspirant below, extract Entities strictly as instructed below
1. First, look for this Education entity type and generate as comma-separated format similar to entity type.
   `id` property of each entity must be alphanumeric and must be unique among the entities. You will be referring this property to define the relationship between entities. NEVER create other entity types that aren't mentioned below. You will have to generate as many entities as needed as per the types below:
    Entity Types:
    label:'Education',id:string,degree:string,university:string,graduation_date:string,score:string,url:string,courses:string //Education Node
2. If you cannot find any information on the entities & relationships above, it is okay to return empty value. DO NOT create fictious data
3. Do NOT create duplicate entities
4. No Skill Entity in the output

Example Output Format:
{
    "entities": [{"label":"Education","id":"education1","degree":"Bachelor of Science","graduationDate":"May 2022","score":"5.0"}]
}

Question: Now, extract entities as mentioned above for the text below -
$ctext

Answer:
"""

In [104]:
sample_que = """Lead Software Developer Lead Software <span class="hl">Developer</span> Lead Software Developer - O'Connor and Associates Montgomery, TX Work Experience Lead Software Developer O'Connor and Associates - Houston, TX October 2016 to Present � Manage acquisition and loading of County Appraisal District (CAD) data for 115+ counties in Texas.� � Manage offshore team of developers for custom built property tax applications.� � Create custom utility applications in Delphi to support business activities.� � Member of company leadership team and Center of Excellence initiative.� � Updated legacy Excel macros to work with Excel 2010+.� � Provide support for several applications in MS Access and C#.� � Create SQL queries for reports for management.� � Setup custom Zoho CRM fields and import sales leads.� � Mentor Junior IT staff� � Design marketing banners and flyers. Data Administrator/Software Developer Property Data Cloud - Houston, TX January 2013 to July 2019 � Manage acquisition, loading, and normalization of CAD data.� � Manage data for over 100 Texas County Appraisal Districts.� � Extract sales, income, cost data from PDF for loading into SQL.� � Database and software support for PDC clients.� � This is a contract, part-time position. Software Developer PCCA / PK Software - Houston, TX January 2013 to September 2016 � Develop Pharmacy software in Delphi 6 and Delphi XE2 with Firebird and SQL Server database backend.� � Convert applications from Delphi 6 to Delphi XE2.� � Took ownership of company website, responsible for updates and changes.� � Main point of contact for customers with database performance issues. Configure customer Firebird installations for maximum performance.� � Main developer and point of contact for integration with TeleManager IVR.� � Manage Axosoft OnTime server.� � Create installation packages with InstallShield.� � Create icons and images for use in PK Software applications and PK Software website. IT Director Commercial Tax Network - Houston, TX April 2006 to December 2012 � Managed computer operations (database, networking, programming, support). As of July, 2009, manage one network/desktop support technician.� � Manage server infrastructure on VMWare ESXi 4.� � Create new internal applications using Delphi 2006 and SQL Server 2008. Applications interface with SQL Server and use a variety of technologies including SMTP, downloading through HTTP, MS Word mail merge, exporting data to MS Excel spreadsheets.� � Completely recoded and redesigned legacy MS Access data system application to Delphi 2006/SQL Server.� � Integrated SmartSearch document management system with custom Delphi/SQL 2008 applications.� � Work closely with company executives to implement and improve business processes.� � Manage the acquisition and loading and normalization of CAD data.� � DBA for SQL Server 2008 database. Manage indexes, stored procedures and triggers to support database stability and performance. Perform ad-hoc queries to support business needs. Import and update data through the use of DTS and SQL queries.� � Query Appraisal District data to provide targeted mailing lists for marketing.� � Administer company email on Exchange Server 2010.� � Interact with vendors (phone, internet, LAN, etc.) to assure quality, reliable services.� � Managed moving of T1, phone, and LAN services to new building with minimal downtime. Provide technical support HP - Houston, TX 2007 to 2010 for 25 - 30 employees in two offices (Houston, TX and Atlanta, GA). Support Windows 7/Windows XP Pro environment as well as MS Office 2003/2007/2010, Quickbooks, IE, Firefox, ArcGIS, HP, Lexmark, and Dell printers and custom designed software.� � Build custom PCs, upgrade and repair PCs.� � Maintain company website (commercialtax.com).� � Work with ArcGIS software to provide GIS mapping solutions. Technical Consultant Kim Lighting - Industry, CA November 2005 to January 2007 � Provide technical consulting for company's website and software applications.� � Provide javascript programming for company's pdf specification sheets. Web Developer/Administrator Danzco Dance Academy - Montgomery, TX August 2004 to December 2006 � Designed and developed www.danzcodanceacademy.com. Site uses object oriented PHP/mySQL to display dynamic content.� � Created custom bulletin board system with PHP/mySQL.� � Converted text file based chat script to use PHP/mySQL.� � Tested new scripts on Apache Web Server. Web Developer/Administrator MCYBA - Magnolia, TX September 2002 to December 2006 Designed and developed website for local youth baseball organization. Site featured dynamic content, such as team schedules, game results, and team articles. Site uses object oriented PHP scripting and mySQL database. Web Developer/Administrator Hit-Away Indoor Sports Facility - Tomball, TX February 2000 to December 2006 � Designed and developed www.hit-away.com. Site used PHP/mySQL to provide dynamic content.� � Provided technical support and consulting services. Software Developer II Desktop Assistance, L.P - Houston, TX October 1999 to November 2005 � Design and develop software applications using Borland Delphi 4, 5, 6 and 7 and several database systems (e.g. Interbase, MS Access, Dbase).� � Designed and developed several key applications in our custom information management system. The first was a messaging and workflow application, similar to MS Outlook. The second was a viewer for viewing our custom documents.� � Developed and maintained client website, www.kimlighting.com.� � Solely responsible for updating Kim Lighting specification sheets for use in their Specification CD software and their website.� � Modified Kim Lighting's PDF product specification sheets to allow for user interaction using JavaScript.� � Handled all technical support for Kim Lighting software and website.� � Helped develop in-house application to automatically build all pages for the Kim Lighting website.� � Developed several small applications that interacted with XML.� � Handled all technical support for our software for the first 1-� years of my employment at Desktop Assistance.� � Developed in-house Development Guidelines and Standards documentation for our current project. Programmer I/II TDCJ - Huntsville, TX 1998 to 1999 � Designed and developed custom software applications for use in the Human Resources headquarters using Borland Delphi 3 and 4, and IBM DB2 database system.� � Communicated with analysts and end-users in several departments to acquire specifications for new applications.� � Helped develop the IBM DB2 database that would be the heart of all Human Resource applications.� � Worked with System Support representatives to help solve user computer problems. (e.g. Software problems, connecting to the database, etc.)� � Was the IS/IT liaison to the web site development group, which consisted of members of each HR department. Internet and Technologies Programmer Tejas Instruments, Inc - The Woodlands, TX 1998 to 1998 � Developed an inventory entry database system using Microsoft Access 97.� � Assisted in the development of Tejas Instruments Oil and Gas Flow Products Inc. and TexaCan Surplus Pipeline Equipment Inc. web sites and Internet endeavors.� � Assisted office staff in use of common software (i.e. Word, Excel) Jr. Programmer Vintage Sports Plaques - Conroe, TX 1997 to 1997 � Developed in-house reporting applications using FoxPro and Dbase.� � Assisted office staff in use of common software (i.e. Word, Excel). Programmer/Analyst Sterling Trust Company - Waco, TX 1995 to 1997 � Developed VAX/VMS data entry and reporting applications using COBOL and DB2 database system.� � Consulted with end-users to obtain specifications for new or existing applications.� � Provided users with a wide range of technical support, from 3rd party software applications, to in-house applications, to basic computer tasks. Education AAS degree in Computer Science Technology in Computer Science Technology Texas State Technical College - Waco, TX 1995 to 1997 AAS degree in design Texas State Technical College Skills MYSQL, Javascript, PHP, CSS, HTML 5, SQL (10+ years), Delphi (10+ years), Wordpress (4 years), Photoshop (8 years), Microsoft Office (10+ years)"""

from string import Template
prompts = [person_prompt_tpl, postion_prompt_tpl, skill_prompt_tpl, edu_prompt_tpl]
import json
results = {"entities": [], "relationships": []}
for p in prompts:
    _prompt = Template(p).substitute(ctext=clean_text(sample_que))
    _extraction = extract_entities_relationships(_prompt, '') #entity_extraction_tuned_model)
    if 'Answer:\n' in _extraction:
        _extraction = _extraction.split('Answer:\n ')[1]
    _extraction = json.loads(_extraction.replace("\'", "'"))
    results["entities"].extend(_extraction["entities"])
    if "relationships" in _extraction:
        results["relationships"].extend(_extraction["relationships"])

In [ ]:
person_id = results["entities"][0]["id"]
for e in results["entities"][1:]:
    if e['label'] == 'Position':
        results["relationships"].append(f"{person_id}|HAS_POSITION|{e['id']}")
    if e['label'] == 'Skill':
        results["relationships"].append(f"{person_id}|HAS_SKILL|{e['id']}")
    if e['label'] == 'Education':
        results["relationships"].append(f"{person_id}|HAS_EDUCATION|{e['id']}")

## Neo4j Cypher Generation

The entities and relationships we got from the LLM have to be transformed to Cypher so we can write them into Neo4j.

In [107]:
import time

def get_prop_str(prop_dict, _id):
    s = []
    for key, val in prop_dict.items():
      if key != 'label' and key != 'id':
         s.append(_id+"."+key+' = "'+str(val).replace('\"', '"').replace('"', '\"')+'"') 
    return ' ON CREATE SET ' + ','.join(s)

def get_cypher_compliant_var(_id):
    return "_"+ re.sub(r'[\W_]', '', _id)

def generate_cypher(in_json):
    e_map = {}
    e_stmt = []
    r_stmt = []
    e_stmt_tpl = Template("($id:$label{id:'$key'})")
    r_stmt_tpl = Template("""
      MATCH $src
      MATCH $tgt
      MERGE ($src_id)-[:$rel]->($tgt_id)
    """)
    for obj in in_json:
      for j in obj['entities']:
          props = ''
          label = j['label']
          id = j['id']
          if label == 'Person':
                id = 'c'+str(time.time_ns())
          elif label == 'Position':
                id = 'p'+str(time.time_ns())
          varname = get_cypher_compliant_var(j['id'])
          stmt = e_stmt_tpl.substitute(id=varname, label=label, key=id)
          e_map[varname] = stmt
          e_stmt.append('MERGE '+ stmt + get_prop_str(j, varname))

      for st in obj['relationships']:
          rels = st.split("|")
          src_id = get_cypher_compliant_var(rels[0].strip())
          rel = rels[1].strip()
          tgt_id = get_cypher_compliant_var(rels[2].strip())
          stmt = r_stmt_tpl.substitute(
              src_id=src_id, tgt_id=tgt_id, src=e_map[src_id], tgt=e_map[tgt_id], rel=rel)
          
          r_stmt.append(stmt)

    return e_stmt, r_stmt

In [109]:
ent_cyp, rel_cyp = generate_cypher([results])

print(ent_cyp, rel_cyp)

['MERGE (_person1:Person{id:\'c1685086177338707569\'}) ON CREATE SET _person1.role = "Lead Software Developer",_person1.description = "Lead Software Developer with over 20 years of experience in software development, database administration, and web development. Expertise in Delphi, PHP, CSS, HTML 5, SQL, Javascript, Wordpress, Photoshop, and Microsoft Office."', 'MERGE (_position1:Position{id:\'p1685086177338741365\'}) ON CREATE SET _position1.title = "Lead Software Developer",_position1.location = "Houston, TX",_position1.startDate = "October 2016",_position1.endDate = "Present"', 'MERGE (_company1:Company{id:\'company1\'}) ON CREATE SET _company1.name = "O\'Connor and Associates"', 'MERGE (_position2:Position{id:\'p1685086177338764631\'}) ON CREATE SET _position2.title = "Data Administrator/Software Developer",_position2.location = "Houston, TX",_position2.startDate = "January 2013",_position2.endDate = "July 2019"', 'MERGE (_company2:Company{id:\'company2\'}) ON CREATE SET _company

## Data Ingestion

You will need a Neo4j AuraDS Pro instance.  You can deploy that on Google Cloud Marketplace [here](https://console.cloud.google.com/marketplace/product/endpoints/prod.n4gcp.neo4j.io).

With that complete, you'll need to install the Neo4j library and set up your database connection.

In [111]:
%pip install --user graphdatascience

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.0/919.0 kB 19.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.7/187.7 kB 27.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 37.9 MB/s eta 0:00:0000:0100:01
  Created wheel for neo4j: filename=neo4j-5.8.1-py3-none-any.whl size=258701 sha256=7a0f2bf53e6440652cb4a1886771cf6c27c68a1e85ea859ba872eb56a2b156e9
  Stored in directory: /home/jupyter/.cache/pip/wheels/89/ee/2a/85f7b50c16580f09d88dcdbfd546db95d5b29b9967a3603ca1
Successfully built neo4j
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [18]:
from graphdatascience import GraphDataScience

In [19]:
# You will need to change these variables
connectionUrl = 'neo4j+s://7c44f42f.databases.neo4j.io'
username = 'neo4j'
password = 'GRvlpOO4-Ozl8iHaV_20ZQqwOaUljgIpnWyKJRmt2Fc'

In [20]:
gds = GraphDataScience(connectionUrl, auth=(username, password))
gds.version()

'2.3.6+19'

Before loading the data, create constraints as below

In [21]:
gds.run_cypher('CREATE CONSTRAINT unique_case_id IF NOT EXISTS FOR (n:Case) REQUIRE n.id IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_person_id IF NOT EXISTS FOR (n:Person) REQUIRE (n.id) IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_symptom_id IF NOT EXISTS FOR (n:Symptom) REQUIRE (n.id) IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_disease_id IF NOT EXISTS FOR (n:Disease) REQUIRE n.id IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_bodysys_id IF NOT EXISTS FOR (n:BodySystem) REQUIRE n.id IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_diag_id IF NOT EXISTS FOR (n:Diagnosis) REQUIRE n.id IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_biological_id IF NOT EXISTS FOR (n:Biological) REQUIRE n.id IS UNIQUE')

""


Ingest the entities

In [22]:
%%time
for e in ent_cyp:
    gds.run_cypher(e)


CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '': expected
  "ALL"
  "ANY"
  "CASE"
  "COLLECT"
  "COUNT"
  "EXISTS"
  "INF"
  "INFINITY"
  "NAN"
  "NONE"
  "REDUCE"
  "SINGLE"
  "allShortestPaths"
  "false"
  "null"
  "shortestPath"
  "true"
  an identifier (line 1, column 62 (offset: 61))
"MERGE (_case1:Case{id:'c1685063809776665649'}) ON CREATE SET"
                                                              ^}

Ingest relationships now

In [ ]:
%%time
for r in rel_cyp:
    gds.run_cypher(r)

This is a helper function to ingest all case sheets inside the `data/` directory

In [ ]:
import glob
def run_pipeline(count=191):
    txt_files = glob.glob("data/case_sheets/*.txt")[0:count]
    print(f"Running pipeline for {len(txt_files)} files")
    failed_files = process_pipeline(txt_files)
    print(failed_files)
    return failed_files

def process_pipeline(files):
    failed_files = []
    for f in files:
        try:
            with open(f, 'r') as file:
                print(f"  {f}: Reading File...")
                data = file.read().rstrip()
                text = clean_text(data)
                print(f"    {f}: Extracting E & R")
                results = extract_entities_relationships(f, text)
                print(f"    {f}: Generating Cypher")
                ent_cyp, rel_cyp = generate_cypher(results)
                print(f"    {f}: Ingesting Entities")
                for e in ent_cyp:
                    gds.run_cypher(e)
                print(f"    {f}: Ingesting Relationships")
                for r in rel_cyp:
                    gds.run_cypher(r)
                print(f"    {f}: Processing DONE")
        except Exception as e:
            print(f"    {f}: Processing Failed with exception {e}")
            failed_files.append(f)
    return failed_files
            
def extract_entities_relationships(f, text):
    start = timer()
    system = "You are a helpful Medical Case Sheet expert who extracts relevant information and store them on a Neo4j Knowledge Graph"
    prompts = [prompt1]
    all_cypher = ""
    results = []
    for p in prompts:
      p = Template(p).substitute(ctext=text)
      res = process_gpt(system, p)
      results.append(json.loads(res))
    end = timer()
    elapsed = (end-start)
    print(f"    {f}: E & R took {elapsed}secs")
    return results

In [ ]:
%%time
failed_files = run_pipeline(200)

If processing failed for some files due to API Rate limit or some other error, you can retry as below

In [ ]:
%%time
failed_files = process_pipeline(failed_files)
failed_files

In [ ]:
results

## Cypher Generation for Consumption

### Tune the model to generate Cypher

In [ ]:
training_data = 'gs://' + bucket_name + '/' + filename
train_steps = 10

vertexai.init(project=project_id, location=location)
model = TextGenerationModel.from_pretrained("text-bison@001")

model.tune_model(
  training_data=training_data,
  train_steps=train_steps,
  tuning_job_location="europe-west4",
  tuned_model_location="us-central1",
)

### Generate Cypher

In [112]:
def english_to_cypher(prompt, tuned_model_name=''):
    try:
        res = run_text_model(project_id, "text-bison@001", 0, 1024, 0.8, 40, prompt, location, tuned_model_name)
        # res = json.loads(res.replace("\'", "'"))
        return res
    except Exception as e:
        print(e)

In [119]:
prompt = """
Context:
You are an expert Neo4j Cypher translator who understands the question in english and convert to Cypher strictly based on the Neo4j Schema provided and the instructions below:
1. Use the Neo4j schema to generate cypher compatible ONLY for Neo4j Version 5
2. Do not use EXISTS, SIZE keywords in the cypher.
3. Use only Nodes and relationships mentioned in the schema while generating the response
4. Reply ONLY in Cypher
5. Always do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Company name use `toLower(c.name) contains 'neo4j'`
6. Candidate node is synonymous to Person.
Now, use this Neo4j schema and Reply ONLY in Cypher when it makes sense.
Schema:
Nodes:
    label:'Person',id:string,role:string,description:string //Person Node
    label:'Position',id:string,title:string,location:string,startDate:string,endDate:string,url:string //Position Node
    label:'Company',id:string,name:string //Company Node
    label:'Skill',id:string,name:string,level:string //Skill Node
    label:'Education',id:string,degree:string,university:string,graduation_date:string,score:string,url:string,courses:string //Education Node
Relationships:
    (:Person)-[:HAS_POSITION]->(:Position)
    (:Position)-[:AT_COMPANY]->(:Company)
    (:Person)-[:HAS_SKILL]->(:Skill)
    (:Person)-[:HAS_EDUCATION]->(:Education)

So, for this question: 'How many experts do I have on Java', you will answer : MATCH (p:Person)-[:HAS_SKILL]->(s:Skill) WHERE toLower(p.name) CONTAINS 'java' AND toLower(p.level) CONTAINS 'expert' RETURN COUNT(p) 
Because:
1. As per schema definition of nodes & relationships above, Person node is related to Skill node via HAS_SKILL relationship.
2. From the schema, Skill has name and levels as properties. Expertise can be checked using `level`
3. Finally, we return the number of persons who match the input criteria using COUNT function


Ouput Format (Strict): //Only code as output. No other text
MATCH (p:Person)-[:HAS_SKILL]->(s:Skill) WHERE toLower(p.name) CONTAINS 'java' AND toLower(p.level) CONTAINS 'expert' RETURN COUNT(p) 

Question:
$ctext

Answer:
"""

que = 'Which universities do most of my candidates come from?'
_prompt = Template(prompt).substitute(ctext=clean_text(que))

cypher = english_to_cypher(_prompt, '')
if 'Answer:\n ' in response:
    cypher = cypher.split('Answer:\n ')[1]
cypher
    

'MATCH (p:Person)-[:HAS_EDUCATION]->(e:Education) RETURN e.university ORDER BY COUNT(e) DESC LIMIT 10'